In [2]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nlp_id.stopword import StopWord 
from nlp_id.postag import PosTag
from nltk.tokenize import sent_tokenize
import torch
from nlp_id.lemmatizer import Lemmatizer 
from googletrans import Translator
import regex
from fuzzysearch import find_near_matches
import datetime
from multiprocessing import  Pool
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pandarallel import pandarallel
import html

## Stock Names Scraping

### Initial Scrape

In [5]:
raw = pd.read_csv("news.csv")
topics = ["Business", "investasi", "market", 
          "Tech", "teknologi", "stocksetup", 
          "Business", "Finance", "IT and Telco", "TEKNO",
          "finansial", "investasi", "keuangan", "telkomindonesia", "Market"]
full_df = raw.loc[raw["topic"].isin(topics), :]

In [6]:
stocks = pd.read_excel("Stock_Names.xlsx", usecols = ["Ticker", "Nama"], keep_default_na = False)


stock_names = stocks.iloc[:,1]
tickers = stocks.iloc[:,0]
zip_iterator = zip(tickers, stock_names)
stock_dict = dict(zip_iterator)
stock_dict["TRUE"] = stock_dict.pop(True)

In [7]:
def search_ticker_title(text):
    res = []
    for names in stock_dict:
            test_title_full = find_near_matches(stock_dict[names], text, max_l_dist=1)
        #### Depan space or blkng non text character 
            pattern = r"\b{temp}\b".format(temp = names)
            test_title_ticker = re.search(pattern, text)
            if test_title_full:
                res.append(names)
            elif test_title_ticker:
                res.append(names) 
    return res

In [8]:
pandarallel.initialize(progress_bar = True)

ticker_title = full_df["title"].apply(str).parallel_apply(search_ticker_title)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
alt_stock = {"BBCA": r"\bBCA\b",
             "BBRI": r"\bBRI\b(?! Syariah| Agroniaga)",
             "AGRO": r"\bBRI Agroniaga\b",
             "BBNI": r"\bBNI (?! Syariah)\b",
             "BRIS": r"\bBRI Syariah\b",
             "ANTM": r"\bAntam\b",
             "AASI": r"\bAstra\b(?! Otoparts| Graphia| Agro)",
             "TLKM": r"\bTelkom",
             "BMRI": r"\bMandiri\b",
             "SMRA": r"\bSumarecon\b",
             "BNII": r"\bMaybank\b",
             "PWON": r"\bPakuwon\b",
             "MNCN": r"\bMNCN\b",
             "UNVR": r"\bUnilever\b",
             "AKRA": r"\bAKR\b",
             "MAYA": r"\bMayapada\b",
             "INTP": r"\bIndocement\b",
             "BDMN": r"\bDanamon\b",
             "BNBR": r"\bBakrie and Brothers\b",
             "BANK": r"\bBank Aladin\b",
             "BBTN": r"\bBTN\b",
             "LIFE": r"\b(Sinarmas MSIG Life|Sinarmas Life)\b",
             "EXCL": r"\bXL\b",
             "BNGA": r"\bCIMB Niaga\b",
             "PNBN": r"\bPanin\b(?! Syariah)",
             "SMGR": r"\bSemen Gresik\b",
             "ACES": r"\bAce Hardware\b",
             "IMAS": r"\bIndomobil\b"}
def find_alternative(text):
    res = []
    for names in alt_stock:
            test_title_ticker = re.search(alt_stock[names], text)
            if test_title_ticker:
                res.append(names)
    return res

In [10]:
alt_ticker_title = full_df["title"].apply(str).parallel_apply(find_alternative)

In [11]:
def combine(first_list, second_list):
    return first_list + list(set(second_list) - set(first_list))

ticker_df = pd.DataFrame({"Original" : ticker_title, "Alternative": alt_ticker_title})

combined_ticker = ticker_df.apply(lambda x: combine(x["Original"], x["Alternative"]), axis = 1)

In [12]:
more_than_one = lambda x: len(x) > 0

In [19]:
ticker_idx = combined_ticker[combined_ticker.apply(more_than_one)]

## Add Translation

In [20]:
%store -r add_trans merged translated_cleaned

past_trans = pd.concat([add_trans, merged, translated_cleaned]).drop_duplicates(keep = "first")

In [21]:
len(past_trans)

140524

In [31]:
def translate_text_en(text, key):
    import os

    # Set environment variables
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key
    
    import six
    
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    result = translate_client.translate(text, target_language= "en")
    
    return result["translatedText"]

In [23]:
title_ticker_trans = full_df \
                .merge(ticker_idx.rename("Ticker"), how = "right", left_index = True, right_index = True) \
                .merge(past_trans.rename("Translation"), how = "left", right_index = True, left_index = True)

In [24]:
new_title_idx = title_ticker_trans["Translation"].isna()
new_title = title_ticker_trans["title"][new_title_idx]

In [25]:
len(title_ticker_trans)

151006

In [26]:
len(new_title)

37184

In [27]:
len(past_trans)

140524

In [28]:
new_title_trans = np.array_split(new_title, 10)
new0 = new_title_trans[0]
new1 = new_title_trans[1]
new2 = new_title_trans[2]
new3 = new_title_trans[3]
new4 = new_title_trans[4]
new5 = new_title_trans[5]
new6 = new_title_trans[6]
new7 = new_title_trans[7]
new8 = new_title_trans[8]
new9 = new_title_trans[9]

In [33]:
translate_text_en("test")

In [29]:
pandarallel.initialize(progress_bar = True)

trans0 = new0.apply(str).parallel_apply(translate_text_en)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


Process ForkPoolWorker-32:
Process ForkPoolWorker-34:
Process ForkPoolWorker-35:
Process ForkPoolWorker-33:
Process ForkPoolWorker-31:
Process ForkPoolWorker-30:
Process ForkPoolWorker-29:
Process ForkPoolWorker-28:
Traceback (most recent call last):
Traceback (most recent call last):


In [ ]:
trans1 = new1.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans2 = new2.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans3 = new3.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans4 = new4.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans5 = new5.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans6 = new6.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans7 = new7.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans8 = new8.apply(str).parallel_apply(translate_text_en)

In [ ]:
trans9 = new9.apply(str).parallel_apply(translate_text_en)

In [ ]:
all_new_trans = pd.concat([trans0,
                           trans1,
                           trans2,
                           trans3,
                           trans4,
                           trans5,
                           trans6,
                           trans7,
                           trans8,
                           trans9])
all_new_trans = pd.concat([past_trans, all_new_trans]).apply(html.unescape).dropna()

In [ ]:
sum(all_new_trans.isna())

In [ ]:
final_df1 = full_df \
            .merge(ticker_idx.rename("Ticker"), how = "right", left_index = True, right_index = True) \
            .merge(all_new_trans.rename("Translation"), how = "left", right_index = True, left_index = True)
%store final_df1

## Ad-Hoc Cleaning

In [ ]:
### Manual Change
def clean_ticker(title, ticker):
    res = ticker.copy()
    if "BANK" in ticker:
        if not (re.search(r"\bbank aladin\b", title.lower()) or re.search(r"\bbank net\b", title.lower()) or re.search(r"-BANK-", title)):
            res.remove("BANK")     
                
    if ("WTON" in res and "WIKA" in res):
        if not(re.search(r"\bwijaya karya\b(?! beton| bangunan)", title.lower()) or re.search(r"\bwika\b(?! beton| bangunan)", title.lower())):
            res.remove("WIKA")
            
    if ("WEGE" in res and "WIKA" in res):
        if not(re.search(r"\bwijaya karya\b(?! beton| bangunan)", title.lower()) or re.search(r"\bwika\b(?! beton| bangunan)", title.lower())):
            res.remove("WIKA")
            
    if ("DPNS" in res and "DUTI" in res):
        if not(re.search(r"\bduta pertiwi\b(?! nusantara)", title.lower()) or re.search(r"\bDUTI\b", title)):
            res.remove("DUTI")
            
    if ("BRMS" in res and "BUMI" in res):
        if not(re.search(r"\bbumi resources\b(?! minerals| mineral)", title.lower()) or re.search(r"\bBUMI\b", title)):
            res.remove("BUMI")

    if ("BTPN" in res and "BBTN" in res):
        if not(re.search(r"\bbank btpn\b", title.lower()) or re.search(r"\bBTPN\b", title)):
            res.remove("BTPN")

    if ("PNBN" in res and "PNBS" in res):
        if not(re.search(r"\bbank panin\b(?! syariah)", title.lower()) or re.search(r"\bPNBN\b", title)):
            res.remove("PNBN")
            
    if ("BLUE" in res and "BIRD" in res):
        if (re.search(r"\bBLUE BIRD\b")):
            res.remove("BLUE")
    
    if "INDO" in res:
        res.remove("INDO")
    if "CITY" in res:
        res.remove("CITY")
    if "LABA" in res:
        res.remove("LABA")
    if "AKSI" in res:
        res.remove("AKSI")
    if "LAND" in res:
        res.remove("LAND")
    
    if "BINA" in res:
        res.remove("BIMA")
    if "CASH" in res:
        res.remove("CASH")
    if "AMAN" in res:
        res.remove("AMAN")
    if "BALI" in ticker:
        res.remove("BALI")
    
    if "CARE" in res:
        res.remove("CARE")
    if "DAYA" in res:
        res.remove("DAYA")
    if "BUDI" in res:
        res.remove("BUDI")
    if "BALI" in ticker:
        res.remove("BALI")
    
    if "FOOD" in res:
        res.remove("FOOD")
    if "LIFE" in res:
        res.remove("LIFE")
        
    return res

In [ ]:
cleaned_ticker = final_df1.apply(lambda x: clean_ticker(x["title"], x["Ticker"]))

In [3]:
final_df1 = pd.read_csv("coba_df.csv")

In [4]:
import ast
final_df1["Final Ticker"] = final_df1["Cleaned Ticker"].apply(ast.literal_eval)
final_df1["Final Ticker"] = final_df1.apply(lambda x: final_tweak(x["Final Ticker"], x["title"]), axis = 1)

In [24]:
final_df1[final_df1["Final Ticker"] == "NASA"]

,Unnamed: 0,title,topic,tag,date,url,language,source,text,Ticker,...,Cleaned Ticker,Date,Time,Year,Month,Day,Hour,Final Ticker,Datetime,Trading Date


## Getting Trading Date

In [18]:
final_df1["Datetime"] = pd.to_datetime(final_df1["date"])


final_df1["Date"] = final_df1["Datetime"].apply(lambda text: text.date())
final_df1["Time"] = final_df1["Datetime"].apply(lambda text: text.time())
final_df1["Year"] = final_df1["Datetime"].apply(lambda text: text.year)
final_df1["Month"] = final_df1["Datetime"].apply(lambda text: text.month)
final_df1["Day"] = final_df1["Datetime"].apply(lambda text: text.day)
final_df1["Hour"] = final_df1["Datetime"].apply(lambda text: text.hour)

In [19]:
price_stock = pd.read_csv("PX_LAST20020101.csv")
price_stock["Date"] = pd.to_datetime(price_stock["Date"], format = "%Y%m%d")
price_stock = price_stock.set_index("Date")

In [20]:
def find_next_date(date_time):
    date = price_stock.index.tolist()
    next_date = date_time + datetime.timedelta(days = 1)
    while next_date not in date:
        next_date += datetime.timedelta(days = 1)
    return next_date

def get_tdate(date_time, hour):
    # Check if date is before trading hour
    # Check next period is in business day
    # If in business date use according to the original rule
    # If not use the
    date = price_stock.index.tolist()
    if hour <= 8:
        if date_time in date:
            return date_time
        else:
            return find_next_date(date_time)
    else:
        return find_next_date(date_time)

In [21]:
pandarallel.initialize(progress_bar = True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [22]:
final_df1["Trading Date"] = final_df1.parallel_apply(lambda x: get_tdate(x["Date"], x["Hour"]), axis = 1)

In [23]:
def final_tweak(ticker , title):
    res = ticker.copy()
    if "AASI" in res:
        res.remove("AASI")
        res.append("ASII")
    if "LIFE" in res:
        res.remove("LIFE")
    if "NASA" in res:
        res.remove("NASA")
    if "WIKA" in res:
        if re.search(r"\bwika gedung\b", title.lower()):
            res.remove("WIKA")
            res.append("WEGE")
    return res


## Remove Rows with more than 1 Stock Name Mentioned

In [25]:
one_idx = final_df1["Final Ticker"].apply(lambda x: len(x) == 1)

In [26]:
final_df2 = final_df1[one_idx]

In [27]:
final_df2["Final Ticker"] = final_df2["Final Ticker"].apply(lambda x: x[0])

<ipython-input-27-308b23a054d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2["Final Ticker"] = final_df2["Final Ticker"].apply(lambda x: x[0])


In [28]:
final_df2 = final_df2.set_index("Unnamed: 0")

## Remove Rows Under IDR 1 T Market Cap

In [31]:
market_cap = pd.read_csv("MKT_CAP 20020101.csv")
market_cap["Date"] = pd.to_datetime(market_cap["Date"], format = "%Y%m%d")
market_cap = market_cap.set_index("Date")

risk_free = pd.read_csv("risk_free.csv")
risk_free["Date"] = pd.to_datetime(risk_free["Date"], format = "%Y%m%d")

def get_market_cap(row):
    ticker_name = row["Final Ticker"]
    date = row["Trading Date"]
    res = market_cap.loc[str(date), ticker_name]
    return res

In [32]:
stock_names = market_cap.columns
final_df3 = final_df2[final_df2["Final Ticker"].isin(stock_names)]

In [33]:
market_cap_df = final_df3.apply(get_market_cap, axis = 1)
idx_mrktcap = market_cap_df >= 10^12

In [34]:
sliced_df = final_df3[idx_mrktcap]

In [35]:
len(sliced_df)

127779

## Remove Unused Columns and Export

In [38]:
final_df3.drop(["date", "Ticker", "Cleaned Ticker"], axis = 1).to_csv("full_df.csv")

In [39]:
sliced_df.drop(["date", "Ticker", "Cleaned Ticker"], axis = 1).to_csv("marketcap_df.csv")